In [ ]:
import os
import json
import mne
import scipy

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
from scipy.signal import hilbert, coherence

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline

mne.set_log_level('WARNING')

In [ ]:
import my_functions as my_fun
import my_visualization_functions as my_vis_fun

In [ ]:
# Define subjects to process
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 31)]  
data_path = "eeg-motor-movementimagery-dataset-1.0.0/files/"

In [ ]:
subjects.remove("S038")

In [ ]:
# Process EEG data
eeg_data = my_fun.process_eeg(subjects=subjects, data_path=data_path, mode="automatic", apply_ica=True)

## Visualization for one subject

In [ ]:
subject = "S001" 
condition = "imagined_right_hand"  # real_left_hand or imagined_left_hand
epochs = eeg_data[subject][condition][0]

In [ ]:
my_vis_fun.plot_raw_eeg(eeg_data[subject][condition][0], subject, condition)

In [ ]:
my_vis_fun.plot_erd_ers(epochs, subject, condition, motor_channels=["C3", "C4"])

## PLV and Coherence Pipeline for one subject

In [ ]:
real_epochs = mne.concatenate_epochs(eeg_data[subject]['real_right_hand'])
imagined_epochs = mne.concatenate_epochs(eeg_data[subject]['imagined_right_hand'])
rest_epochs = mne.concatenate_epochs(eeg_data[subject]['rest'])

In [ ]:
plv_real = my_fun.compute_plv_matrix(real_epochs)
plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
plv_rest = my_fun.compute_plv_matrix(rest_epochs)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real, real_epochs.ch_names, title="PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined, imagined_epochs.ch_names, title="PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_rest, rest_epochs.ch_names, title="PLV - Rest")

In [ ]:
my_vis_fun.plot_plv_difference(plv_real, plv_imagined, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real, plv_rest, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

In [ ]:
my_vis_fun.plot_motor_plv_difference(plv_real, plv_imagined, real_epochs.ch_names, title="PLV (Motor Cortex) - Real Right Hand")

## PLV and Coherence Pipeline for multiple subjects

In [ ]:
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 31)]  

plv_results = {"real": [], "imagined": [], "rest": []}

eeg_data_all = {}  # <-- Store all subjects

for subject in subjects:
    print(f"\n🚀 Processing {subject}...")
    
    eeg_data = my_fun.process_eeg([subject], data_path="eeg-motor-movementimagery-dataset-1.0.0/files/", mode="automatic", apply_ica=True)

    # Merge subject's data into master dict
    eeg_data_all.update(eeg_data)

    # Compute PLV matrix
    real_epochs = mne.concatenate_epochs(eeg_data[subject]["real_right_hand"])
    imagined_epochs = mne.concatenate_epochs(eeg_data[subject]["imagined_right_hand"])
    rest_epochs = mne.concatenate_epochs(eeg_data[subject]["rest"])

    plv_real = my_fun.compute_plv_matrix(real_epochs)
    plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
    plv_rest = my_fun.compute_plv_matrix(rest_epochs)

    plv_results["real"].append(plv_real)
    plv_results["imagined"].append(plv_imagined)
    plv_results["rest"].append(plv_rest)


In [ ]:
plv_real_group = np.mean(plv_results["real"], axis=0)
plv_imagined_group = np.mean(plv_results["imagined"], axis=0)
plv_rest_group = np.mean(plv_results["rest"], axis=0)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real_group, real_epochs.ch_names, title="Group Average PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Rest")

my_vis_fun.plot_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real_group, plv_rest_group, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

#my_vis_fun.plot_motor_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)

In [ ]:
# Define subjects and channel pairs
#subjects = ["S001", "S002", "S005"]
conditions = {"real": "real_right_hand", "imagined": "imagined_right_hand", "rest": "rest"} 

channel_pairs = [
    ("C3", "C4"),  # Primary motor cortex
    ("C1", "C2"),  # Central midline
    ("C5", "C6"),  # Central lateral
    ("FC3", "FC4"),  # Frontal-motor
    ("Cz", "CPz"),  # Midline execution
    ("CP3", "CP4"),  # Parietal-motor
    ("Fz", "Cz"),  # Frontal-central connection
    ("Fp1", "Fp2"),  # Frontal
    ("AF3", "AF4"),  # Anterior frontal
    ("O1", "O2"),  # Occipital 
    ("PO7", "PO8"),  # Parietal
    ("Fp1", "PO8"),   # Prefrontal
    ("T9", "T10"),  # Temporal
]

# Analyze
df_plv_coh = my_fun.analyze_pairwise_plv_coherence(subjects, eeg_data_all, conditions, channel_pairs)

# Plot!
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="PLV Mean")
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="Coherence Mean")

In [ ]:
df_stats = my_fun.paired_ttest_plv(df_plv_coh, metric="PLV Mean")
my_fun.report_paired_ttests(df_stats)

In [ ]:
def plot_normalized_plv_coherence(df, metric="PLV Mean", figsize=(14, 7)):
    """
    Plot PLV or Coherence normalized to the Rest condition.
    Rest condition is not shown but serves as the baseline (value 1.0).
    
    Parameters:
    -----------
    df : DataFrame
        Original PLV/Coherence DataFrame
    metric : str
        Metric to normalize: e.g., "PLV Mean"
    figsize : tuple
        Size of the matplotlib figure
    """
    if metric not in ["PLV Mean", "PLV Max", "Coherence Mean", "Coherence Max"]:
        raise ValueError("Invalid metric.")

    df_pivot = df.pivot_table(index=["Subject", "Channel Pair"], columns="Condition", values=metric)
    df_pivot = df_pivot.dropna(subset=["Rest"])

    # Normalize Real and Imagined
    df_norm = df_pivot.copy()
    df_norm["Real"] = df_norm["Real"] / df_norm["Rest"]
    df_norm["Imagined"] = df_norm["Imagined"] / df_norm["Rest"]

    df_long = df_norm[["Real", "Imagined"]].reset_index().melt(
        id_vars=["Subject", "Channel Pair"],
        var_name="Condition",
        value_name=metric
    )

    # Plot
    plt.figure(figsize=figsize)
    ax = sns.barplot(data=df_long, x="Channel Pair", y=metric, hue="Condition", palette="Set2")

    # Reference line for Rest (value = 1)
    plt.axhline(1.0, color='gray', linestyle='--', linewidth=1.2, label="Rest Baseline")

    # Labels
    plt.xlabel("Electrode Pair")
    plt.ylabel(f"Normalized {metric} (÷ Rest)")
    plt.title(f"Normalized {metric} (Real/Imagined divided by Rest)")
    plt.xticks(rotation=45)
    plt.legend(title="Condition")
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_normalized_plv_coherence(df_plv_coh)

In [ ]:
def plot_ttest_summary(df_stats, alpha=0.05, figsize=(10, 6)):
    """
    Graphical summary of t-test p-values with significance indication.
    
    Parameters:
    -----------
    df_stats : DataFrame
        DataFrame from paired_ttest_plv
    alpha : float
        Significance threshold
    figsize : tuple
        Size of the figure
    """
    import matplotlib.colors as mcolors

    pval_df = df_stats[["Channel Pair", "p Real vs Rest", "p Real vs Imagined"]].copy()
    pval_df.set_index("Channel Pair", inplace=True)

    # Replace missing with 1 (non-significant)
    pval_matrix = pval_df.fillna(1.0)

    # Create mask for significance
    sig_mask = pval_matrix <= alpha

    # Custom colormap: light = non-sig, dark = sig
    cmap = sns.light_palette("crimson", as_cmap=True)

    plt.figure(figsize=figsize)
    ax = sns.heatmap(
        pval_matrix,
        cmap=cmap,
        annot=pval_matrix.applymap(lambda x: f"{x:.4f}"),
        fmt="",
        linewidths=0.5,
        linecolor='gray',
        cbar_kws={"label": "p-value"},
        vmin=0, vmax=1
    )

    # Overlay asterisk for significant results
    for y in range(pval_matrix.shape[0]):
        for x in range(pval_matrix.shape[1]):
            if sig_mask.iloc[y, x]:
                ax.text(x + 0.5, y + 0.5, "*", ha='center', va='center', fontsize=18, color='black')

    ax.set_title(f"T-test P-Values (Significant p ≤ {alpha})\n* = statistically significant")
    ax.set_xlabel("Comparison")
    ax.set_ylabel("Channel Pair")
    plt.tight_layout()
    plt.show()


In [ ]:
plot_ttest_summary(df_stats)

# Time Frequency Representation Analysis

In [ ]:
# For each condition (real, imagined, rest)
freqs = np.arange(6, 30, 1)  # 5-40 Hz range
n_cycles = freqs / 2

power_real = my_fun.tfr_morlet(real_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_imagined = my_fun.tfr_morlet(imagined_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_rest = my_fun.tfr_morlet(rest_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)

In [ ]:
# Plot time-frequency plots for key channels (C3, C4)
power_real.plot_joint(title='Real Movement', picks=['Cz', 'CPz'])
power_imagined.plot_joint(title='Imagined Movement', picks=['Cz', 'CPz'])
power_rest.plot_joint(title='Rest', picks=['Cz', 'CPz'])

# Classification / Decoding Part

In [ ]:
# 1. Run classification on both condition pairs
group_results, individual_results = my_fun.classify_condition_pairs(eeg_data, subjects)

# 2. Visualize the comparison
summary = my_vis_fun.visualize_comparison(group_results, individual_results)

# Frequency Bands and Time Window Analysis

In [ ]:
# Run the frequency band analysis
band_results, band_names = my_fun.analyze_frequency_bands(eeg_data, subjects)

In [ ]:
# Run the time window analysis
window_results, window_names = my_fun.analyze_time_windows(eeg_data, subjects)

# RDM Analysis